
**Import libraries**

In [9]:
import string; import time; import scipy
import scipy.sparse
import scipy.sparse.linalg

In [10]:
import sys
sys.path.append('../../librairies')

 Choose between the Fortran or the Python librairie:

In [11]:
#import silex_lib_tet4_fortran as silex_lib_elt
import silex_lib_tet4_python as silex_lib_elt
import silex_lib_gmsh

install mumps to solve the system more quickly or use petsc

In [2]:
import mumps
#from petsc4py import PETsc

ImportError: /opt/conda/lib/python3.9/site-packages/mumps/_dmumps.cpython-39-x86_64-linux-gnu.so: undefined symbol: mpi_reduce_

In [13]:
print ("SILEX CODE - computation of a with tet4 elements")
#############################################################################

SILEX CODE - computation of a with tet4 elements


In [14]:
tic = time.process_time()


#############################################################################
#      USER PART: Import mesh, boundary conditions and material
#############################################################################

Input mesh: define the name of the mesh file (*.msh)

In [15]:
MeshFileName='piston-tet4'

Output result file: define the name of the result file (*.msh)

In [16]:
ResultsFileName='Results_piston_tet4'

choose the element type

In [17]:
eltype=4

choose geometry dimension

In [18]:
ndim=3

choose the results in the results file

In [19]:
flag_write_fields=0

read the mesh from gmsh

In [20]:
nodes=silex_lib_gmsh.ReadGmshNodes(MeshFileName+'.msh',ndim)
elements,Idnodes=silex_lib_gmsh.ReadGmshElements(MeshFileName+'.msh',eltype,5)

read surfaces where to impose boundary conditions

In [21]:
elementsS1,IdnodeS1=silex_lib_gmsh.ReadGmshElements(MeshFileName+'.msh',2,1)
elementsS2,IdnodeS2=silex_lib_gmsh.ReadGmshElements(MeshFileName+'.msh',2,2)
elementsS3,IdnodeS3=silex_lib_gmsh.ReadGmshElements(MeshFileName+'.msh',2,3)
elementsS4,IdnodeS4=silex_lib_gmsh.ReadGmshElements(MeshFileName+'.msh',2,4)

write the surface mesh in a gmsh-format file to verify if its correct<br>
ilex_lib_gmsh.WriteResults(ResultsFileName+'Volum',nodes,elements,4)<br>
ilex_lib_gmsh.WriteResults(ResultsFileName+'surf1',nodes,elementsS1,2)<br>
ilex_lib_gmsh.WriteResults(ResultsFileName+'surf2',nodes,elementsS2,2)<br>
ilex_lib_gmsh.WriteResults(ResultsFileName+'surf3',nodes,elementsS3,2)<br>
ilex_lib_gmsh.WriteResults(ResultsFileName+'surf4',nodes,elementsS4,2)

Define material

In [22]:
Young  = 200000.0
nu     = 0.3

Boundary conditions

In [23]:
IdNodesFixed_x=IdnodeS3
IdNodesFixed_y=IdnodeS1
IdNodesFixed_z=IdnodeS2

compute external forces from pressure

In [24]:
press = -10 #MPa
# give the direction of the surfacic load:
#          if [0.0,0.0,0.0] then the local normal to the surface is used
#          otherwise, the direction is normalized to 1
direction = [0.0,0.0,0.0]
F = silex_lib_elt.forceonsurface(nodes,elementsS4,press,direction)

In [26]:
toc = time.process_time()
print ("Time for the user part:",toc-tic)

Time for the user part: 0.3851462830000001


###########################################################################<br>
     EXPERT PART<br>
###########################################################################<br>
     initialisations<br>
###########################################################################<br>
get number of nodes, dof and elements from the mesh

In [27]:
nnodes = nodes.shape[0]
ndof   = nnodes*ndim
nelem  = elements.shape[0]
print ("Number of nodes:",nnodes)
print ("Number of elements:",nelem)

Number of nodes: 8632
Number of elements: 36902


define fixed dof

In [28]:
Fixed_Dofs = scipy.hstack([(IdNodesFixed_x-1)*3,(IdNodesFixed_y-1)*3+1,(IdNodesFixed_z-1)*3+2])

/tmp/ipykernel_24/3193242712.py:1: DeprecationWarning: scipy.hstack is deprecated and will be removed in SciPy 2.0.0, use numpy.hstack instead
  Fixed_Dofs = scipy.hstack([(IdNodesFixed_x-1)*3,(IdNodesFixed_y-1)*3+1,(IdNodesFixed_z-1)*3+2])


define free dof

In [29]:
SolvedDofs = scipy.setdiff1d(range(ndof),Fixed_Dofs)

/tmp/ipykernel_24/1426069751.py:1: DeprecationWarning: scipy.setdiff1d is deprecated and will be removed in SciPy 2.0.0, use numpy.setdiff1d instead
  SolvedDofs = scipy.setdiff1d(range(ndof),Fixed_Dofs)


initialize displacement vector

In [30]:
Q=scipy.zeros(ndof)

/tmp/ipykernel_24/1673119538.py:1: DeprecationWarning: scipy.zeros is deprecated and will be removed in SciPy 2.0.0, use numpy.zeros instead
  Q=scipy.zeros(ndof)


###########################################################################<br>
     compute stiffness matrix<br>
###########################################################################

In [31]:
tic0 = time.process_time()
tic = time.process_time()
Ik,Jk,Vk=silex_lib_elt.stiffnessmatrix(nodes,elements,[Young,nu])
toc = time.process_time()

In [32]:
K=scipy.sparse.csr_matrix( (Vk,(Ik,Jk)), shape=(ndof,ndof) ,dtype=float)
print ("Time to compute the stiffness matrix:",toc-tic)

Time to compute the stiffness matrix: 2.847690774


###########################################################################<br>
      Solve the problem<br>
###########################################################################

In [35]:
tic = time.process_time()
%timeit Q[SolvedDofs] = scipy.sparse.linalg.spsolve(K[SolvedDofs,:][:,SolvedDofs],F[SolvedDofs])
# install mumps to improve the computational time
# import mumps
#Q[SolvedDofs] = mumps.spsolve(K[SolvedDofs,:][:,SolvedDofs],F[SolvedDofs])
toc = time.process_time()
print ("Time to solve the problem:",toc-tic)

2.15 s ± 100 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Time to solve the problem: 138.64462278500002


###########################################################################<br>
      compute smooth stress and error in elements<br>
###########################################################################

In [ ]:
tic = time.process_time()

In [ ]:
SigmaElem,SigmaNodes,EpsilonElem,EpsilonNodes,ErrorElem,ErrorGlobal=silex_lib_elt.compute_stress_strain_error(nodes,elements,[Young,nu],Q)

In [ ]:
toc = time.process_time()
print ("Time to compute stress and error:",toc-tic)
print ("The global error is:",ErrorGlobal)
print ("Total time for the computational part:",toc-tic0)

###########################################################################<br>
        Write results to gmsh format<br>
###########################################################################

In [ ]:
tic = time.process_time()

displacement written on 3 columns:

In [ ]:
disp=scipy.zeros((nnodes,ndim))
disp[range(nnodes),0]=Q[list(range(0,ndof,3))]
disp[range(nnodes),1]=Q[list(range(1,ndof,3))]
disp[range(nnodes),2]=Q[list(range(2,ndof,3))]

external load written on 3 columns:

In [ ]:
load=scipy.zeros((nnodes,ndim))
load[range(nnodes),0]=F[list(range(0,ndof,3))]
load[range(nnodes),1]=F[list(range(1,ndof,3))]
load[range(nnodes),2]=F[list(range(2,ndof,3))]

In [ ]:
if flag_write_fields==0:
    fields_to_write=[ [disp,'nodal',ndim,'displacement'],
                      [SigmaElem[range(nelem),[6]],'elemental',1,'Sigma V.M.'],
                      [SigmaNodes[range(nnodes),[6]],'nodal',1,'Smooth Sigma V.M.'],
                      [ErrorElem,'elemental',1,'error'],
                      ]

In [ ]:
if flag_write_fields==1:
    fields_to_write=[ [disp,'nodal',ndim,'displacement'],
                      [load,'nodal',ndim,'Force'],
                      [SigmaElem[range(nelem),[0]],'elemental',1,'Sigma 11'],
                      [SigmaElem[range(nelem),[1]],'elemental',1,'Sigma 22'],
                      [SigmaElem[range(nelem),[2]],'elemental',1,'Sigma 33'],
                      [SigmaElem[range(nelem),[3]],'elemental',1,'Sigma 23'],
                      [SigmaElem[range(nelem),[4]],'elemental',1,'Sigma 13'],
                      [SigmaElem[range(nelem),[5]],'elemental',1,'Sigma 12'],
                      [SigmaElem[range(nelem),[6]],'elemental',1,'Sigma V.M.'],
                      [SigmaNodes[range(nnodes),[0]],'nodal',1,'Smooth Sigma 11'],
                      [SigmaNodes[range(nnodes),[1]],'nodal',1,'Smooth Sigma 22'],
                      [SigmaNodes[range(nnodes),[2]],'nodal',1,'Smooth Sigma 33'],
                      [SigmaNodes[range(nnodes),[3]],'nodal',1,'Smooth Sigma 23'],
                      [SigmaNodes[range(nnodes),[4]],'nodal',1,'Smooth Sigma 13'],
                      [SigmaNodes[range(nnodes),[5]],'nodal',1,'Smooth Sigma 12'],
                      [SigmaNodes[range(nnodes),[6]],'nodal',1,'Smooth Sigma V.M.'],
                      [EpsilonElem[range(nelem),[0]],'elemental',1,'Epsilon 11'],
                      [EpsilonElem[range(nelem),[1]],'elemental',1,'Epsilon 22'],
                      [EpsilonElem[range(nelem),[2]],'elemental',1,'Epsilon 33'],
                      [EpsilonElem[range(nelem),[3]]/2.0,'elemental',1,'Epsilon 23'],
                      [EpsilonElem[range(nelem),[4]]/2.0,'elemental',1,'Epsilon 13'],
                      [EpsilonElem[range(nelem),[5]]/2.0,'elemental',1,'Epsilon 12'],
                      [EpsilonNodes[range(nnodes),[0]],'nodal',1,'Smooth Epsilon 11'],
                      [EpsilonNodes[range(nnodes),[1]],'nodal',1,'Smooth Epsilon 22'],
                      [EpsilonNodes[range(nnodes),[2]],'nodal',1,'Smooth Epsilon 33'],
                      [EpsilonNodes[range(nnodes),[3]]/2.0,'nodal',1,'Smooth Epsilon 23'],
                      [EpsilonNodes[range(nnodes),[4]]/2.0,'nodal',1,'Smooth Epsilon 13'],
                      [EpsilonNodes[range(nnodes),[5]]/2.0,'nodal',1,'Smooth Epsilon 12'],
                      [ErrorElem,'elemental',1,'error'],
                      ]

write the mesh and the results in a gmsh-format file

In [ ]:
silex_lib_gmsh.WriteResults(ResultsFileName,nodes,elements,eltype,fields_to_write)

In [ ]:
toc = time.process_time()
print ("Time to write results:",toc-tic)
print ("----- END -----")